# CHATBOTS - for everyone and for every cause

In [29]:

import numpy as np
import tensorflow as tf
import tflearn
import random

from tensorflow.python.framework import ops

import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()


import json
import pickle
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
print("Loading Dataset.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Loading Dataset.....


In [31]:
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
    
        w = nltk.word_tokenize(pattern)
 
        words.extend(w)
    
        documents.append((w, intent['tag']))
    
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [32]:
print("Stemming, please stand by")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, please stand by
28 documents
10 classes ['goodbye', 'greeting', 'hours', 'jamal', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
49 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'nigg', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [33]:
print("Creating model")
training = []
output = []
print("By retr0")
output_empty = [0] * len(classes)

print("training set ready..")
for doc in documents:
  
    bag = []
  
    pattern_words = doc[0]
    
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
   
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

  
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating model
By retr0
training set ready..


In [34]:

random.shuffle(training)
training = np.array(training)


train_x = list(training[:,0])
train_y = list(training[:,1])

ops.reset_default_graph()

print("graph will not be displayed due to untrained set")


graph will not be displayed due to untrained set


In [35]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Training....


In [36]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [37]:
print("Training the Model please wait....")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model")
model.save('model.tflearn')


Training Step: 3999  | total loss: 0.09094 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.09094 - acc: 1.0000 -- iter: 24/28
Training Step: 4000  | total loss: 0.11188 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.11188 - acc: 1.0000 -- iter: 28/28
--
Saving the Model


In [38]:
print("Indents saved too...")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Indents saved too...


In [39]:
print("Loading.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
print("Loading Model......")

model.load('./model.tflearn')

Loading.....
Loading Model......


In [40]:
print("All set 100% ready to go")

def clean_up_sentence(sentence):
  
    sentence_words = nltk.word_tokenize(sentence)
 
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("known indents: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("E-THRESHOLD = 0.25")

def classify(sentence):
  
    results = model.predict([bow(sentence, words)])[0]

    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
   
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)

    if results:
 
        while results:
            for i in intents['intents']:
          
                if i['tag'] == results[0][0]:
            
                    return print(random.choice(i['responses']))

            results.pop(0)

All set 100% ready to go
E-THRESHOLD = 0.25


In [41]:
print("enter your name")
x = input()

enter your name
top g


In [42]:
while True:
    
    input_data = input(x+"-")
    answer = response(input_data)
    answer

top g-hi
Hi there, how can I help?
top g-good bye my nigger 
See you later, thanks for visiting
top g-jamal
Hello, thanks for visiting
top g-Nigga
Welcome soldier!


KeyboardInterrupt: ignored